In [3]:
import pandas as pd

import re

import requests
import pdfplumber

from bs4 import BeautifulSoup
import os

# Checking out the website

In [45]:
# Contains the name of the reports that have been updated
def update_reports():
    return os.listdir('./reports')
REPORTS = update_reports()
list_to_update = []

In [95]:
def beautify_page(url="https://www.transportation.gov/individuals/aviation-consumer-protection/air-travel-consumer-reports-2020"):
    page = requests.get(url)
    if page.status_code == 200:
        soup = BeautifulSoup(page.content, 'html.parser')
        print('Connection Successful!')
        print(url)
        return soup
    else:
        print('Connection Failure!')
        print(f'Status Code: {page.status_code}')

In [98]:
soup = beautify_page()

Connection Successful!
https://www.transportation.gov/individuals/aviation-consumer-protection/air-travel-consumer-reports-2020


In [107]:
def get_all_pdf(soup):
    list_to_update = []
    report = soup.find_all('div', class_='mb-4 clearfix')
    
    for a in report[0].find_all('a', href=True):
        sub_link = a['href']
        if 'individual' in sub_link:
            if not sub_link.startswith('https'):
                sub_link = 'https://www.transportation.gov' + sub_link
            sub_page = beautify_page(sub_link)

            list_to_update.append(sub_page.find(class_='file').find('a')['href'])
            
    return list_to_update

In [101]:
list_to_update = get_all_pdf(soup)

Connection Successful!
https://www.transportation.gov/individuals/aviation-consumer-protection/december-2020-air-travel-consumer-report
Connection Successful!
https://www.transportation.gov/individuals/aviation-consumer-protection/november-2020-air-travel-consumer-report
Connection Successful!
https://www.transportation.gov/individuals/aviation-consumer-protection/october-2020-air-travel-consumer-report
Connection Successful!
https://www.transportation.gov/individuals/aviation-consumer-protection/september-2020-air-travel-consumer-report
Connection Successful!
https://www.transportation.gov/individuals/aviation-consumer-protection/august-2020-air-travel-consumer-report
Connection Successful!
https://www.transportation.gov/individuals/aviation-consumer-protection/july-2020-air-travel-consumer-report
Connection Successful!
https://www.transportation.gov/individuals/aviation-consumer-protection/june-2020-air-travel-consumer-report
Connection Successful!
https://www.transportation.gov/indi

# Downloading and Openining the file

In [124]:
# Source: https://www.youtube.com/watch?v=eTz3VZmNPSE
def download_pdf(url):
    # Extracts the last part of the URL to be used as the name of the file
    local_filename = url.split('/')[-1].replace('%','')
    
    if local_filename not in REPORTS:
        with requests.get(url) as r:
            with open(f'reports/{local_filename}', 'wb') as f:
                f.write(r.content)
                
        # updates report files in the directory
        return f'reports/{local_filename}'
    else:
        print(f'Already in the database - {local_filename}')
        return False

In [128]:
# URL = 'https://www.transportation.gov/sites/dot.gov/files/2021-02/February_%202021%20ATCR.pdf'
URL = 'https://www.transportation.gov/sites/dot.gov/files/2020-01/january-2020-atcr-1-16-2020.pdf'

In [129]:
filename = download_pdf(URL)

# Dictionaries

In [34]:
months_dict = {'january': 1, 'february': 2, 'march': 3, 'april':4, 'may': 5, 
               'june': 6, 'july': 7, 'august': 8, 'september': 9, 'october': 10,
               'november': 11, 'december': 12, 'jan': 1, 'feb': 2, 'mar': 3, 'apr':4,
               'jun': 6, 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}

In [35]:
quarters_dict = {'january': 1, 'april':2, 'july': 3, 'october': 4,
                 'jan': 1, 'apr':2, 'jul': 3, 'oct': 4}

In [36]:
features_dict = {'mishandled_baggage': ['Number of Bags Enplaned',
                                      'Number of Bags Mishandled',
                                      'Number of Bags Mishandled Per 1000 Enplaned'],
                'mishandled_ws': ['Number of Wheelchairs and Scooters Enplaned',
                                  'Number of Wheelchairs and Scooters Mishandled',
                                  'Percent of wheelchairs and Scooters Mishandled'],
                'denied_boarding': ['Voluntary', 'Involuntary', 'Enplaned', 'Involuntary DB Per 10000 Passengers']}

# Finding the desired page number

In [132]:
def find_operating_page_numbers(filename):
    with pdfplumber.open(filename) as pdf:
        page = pdf.pages[1] # page 41 is missing baggage information
        text = page.extract_text()
    op_re_exp = r'(Operating Carrier (\(Monthly\)|\(Quarterly\)) \s*\d{1,})|(Reporting Carrier(\s*|\s\(Quarterly\)\s*)\d{1,})'
    re_operating_pages = re.compile(op_re_exp)
    
    operating_pages = [] # mishandled baggage / mishandled wheelchairs
    for line in text.split('\n'):
        if re_operating_pages.search(line):
            operating_pages.append(int(list(filter(lambda x: x!='', line.split(' ')))[-1]))
    if len(operating_pages) < 3:
        print('There are only 3 operating pages!')
    return operating_pages

# Mishandled Baggages

In [141]:
def get_table_values_monthly(filename, page_num):
    with pdfplumber.open(filename) as pdf:
        page = pdf.pages[page_num - 1] 
        text = page.extract_text()

    # Regex
    re_month = re.compile(r'^[A-Za-z]*.\d{2,}') # Finds the month/year
    re_new_rank = re.compile(r'^\d{1,}\s*[A-Z].*') # finds indices
    re_carrier_name = re.compile(r'[A-Z].*[A-Z]') # Carrier Name

    # instantiate lists for values
    values = []
    carrier_names = []
    month = None
    for i, line in enumerate(text.split('\n')):
        # Extracts month infomation
        if not month and re_month.search(line):
            if '-' in line:
                month = months_dict[line.split('-')[0].lower()]
                year = int('20'+line.split('-')[1][:2])
            else:
                month = months_dict[line.split(' ')[0].lower()]
                year = int(line.split(' ')[1])
        if re_new_rank.match(line):
            try:
                carrier_names.append(re_carrier_name.search(line)[0])

            except:
                print(f'An error has occured while parsing through a line. [Carrier Name] Line number {i}')
                print(line)
            try:
                # Filters out all empty strings from the list
                vals = list(filter(lambda x: x!='', line.split('  ')))[2:5]

                # Converts string to numerical values int or float
                vals = [int(x.replace(',', '')) if i!=len(vals)-1 else float(x) for i, x in enumerate(vals)]


                values.append(vals)
            except:
                print(f'An error has occured while parsing through a line. [MB or WS] Line number {i}')
                print(line)
    return carrier_names, values, month, year

In [62]:
carrier_names, values, month, year = get_table_values_monthly(filename, operating_pages[0])
# Creating DataFrame

df_mb = pd.DataFrame(values)
df_mb.columns = features_dict['mishandled_baggage']
df_mb['Carrier'] = carrier_names
df_mb['Month'] = month
df_mb['Year'] = year

,Number of Bags Enplaned,Number of Bags Mishandled,Number of Bags Mishandled Per 1000 Enplaned,Month,Year
Carrier,,,,,
EXPRESSJET AIRLINES,19057,31,1.63,4,2020
ALLEGIANT AIR,11823,25,2.11,4,2020
SOUTHWEST AIRLINES,443638,1150,2.59,4,2020
HAWAIIAN AIRLINES,14848,40,2.69,4,2020
ENDEAVOR AIR,31112,94,3.02,4,2020
MESA AIRLINES,51826,164,3.16,4,2020
PSA AIRLINES,96764,334,3.45,4,2020
DELTA AIR LINES,298593,1043,3.49,4,2020
SKYWEST AIRLINES,159489,607,3.81,4,2020


# Mishandled Wheelchairs and Scooters

In [426]:
carrier_names, values, month, year = get_table_values_monthly(filename, operating_pages[1])

df_ws = pd.DataFrame(values)
df_ws.columns = features_dict['mishandled_ws']
df_ws['Carrier'] = carrier_names
df_ws['Month'] = month
df_ws['Year'] = year

df_ws.set_index('Carrier')

,Number of Wheelchairs and Scooters Enplaned,Number of Wheelchairs and Scooters Mishandled,Percent of wheelchairs and Scooters Mishandled,Month,Year
Carrier,,,,,
FRONTIER AIRLINES,221,0,0.00,4,2020
JETBLUE AIRWAYS,127,0,0.00,4,2020
ALLEGIANT AIR,50,0,0.00,4,2020
REPUBLIC AIRWAYS,34,0,0.00,4,2020
ENDEAVOR AIR,30,0,0.00,4,2020
HAWAIIAN AIRLINES,29,0,0.00,4,2020
EXPRESSJET AIRLINES,14,0,0.00,4,2020
SKYWEST AIRLINES,218,1,0.46,4,2020
DELTA AIR LINES,662,5,0.76,4,2020


In [144]:
def get_table_values_quarterly(filename, page_num): 
    with pdfplumber.open(filename) as pdf:
        page = pdf.pages[page_num] 
        text = page.extract_text()

    # Regex
    re_month = re.compile(r'^[A-Za-z]*.-.[A-Za-z]*.\d{2,}') # Finds the month/year
    re_new_rank = re.compile(r'^\d{1,}\s*[A-Z].*') # finds indices
    re_carrier_name = re.compile(r'[A-Z].*[A-Z]') # Carrier Name

    # instantiate lists for values
    values = []
    carrier_names = []
    quarter = None
    for i, line in enumerate(text.split('\n')):
        # Extracts time infomation
        if not quarter and re_month.search(line):
            quarter = quarters_dict[line.split(' ')[0].lower()]
#             year = int(line.split(' ')[3])
        if re_new_rank.match(line):
            try:
                carrier_names.append(re_carrier_name.search(line)[0])

            except:
                print(f'An error has occured while parsing through a line. [Carrier Name] Line number {i}')
                print(line)
            try:
                # Filters out all empty strings from the list
                vals = list(filter(lambda x: x!='', line.split('  ')))[2:6]

                vals = [int(x.replace(',', '')) if i!=len(vals)-1 else float(x) for i, x in enumerate(vals)]

                values.append(vals)
            except:
                print(f'An error has occured while parsing through a line. [Denied Boarding] Line number {i}')
                print(line)
    return carrier_names, values, quarter

In [448]:
carrier_names, values, quarter, year = get_table_values_quarterly(filename, 41)

df_db = pd.DataFrame(values)
df_db.columns = features_dict['denied_boarding']
df_db['Carrier'] = carrier_names
df_db['Quarter'] = quarter
df_db['Year'] = year

df_db.set_index('Carrier')

,Voluntary,Involuntary,Enplaned,Involuntary DB Per 10000 Passengers,Quarter,Year
Carrier,,,,,,
DELTA AIR LINES,16399,0,27870231,0.00,1,2020
ALLEGIANT AIR,272,0,3197140,0.00,1,2020
ENDEAVOR AIR,3775,0,2964249,0.00,1,2020
EXPRESSJET AIRLINES,669,0,1239445,0.00,1,2020
JETBLUE AIRWAYS,250,2,7303799,0.00,1,2020
UNITED AIR LINES,2842,7,18412040,0.00,1,2020
HAWAIIAN AIRLINES,125,1,2185709,0.00,1,2020
SOUTHWEST AIRLINES,2355,112,29539107,0.04,1,2020
ALASKA AIRLINES,1515,30,6420719,0.05,1,2020


# Iterating through different files

In [133]:
REPORTS = update_reports()

In [146]:
REPORTS = update_reports()

# Instantiate dataframes
df_mb = pd.DataFrame() # missing baggages
df_ws = pd.DataFrame() # missing wheelchair/scooters
df_db = pd.DataFrame() # denied boarding

# Gets the DOT page with reports
soup = beautify_page()

# Gets list of URL that are available on DOT site
list_to_update = get_all_pdf(soup)

for url in list_to_update[::-1]:
    filename = download_pdf(url)
    print(filename)
    if filename:
        operating_pages = find_operating_page_numbers(filename)
        print(operating_pages)
        
        ## Mishandling Baggages ##
        print('Processing... Mishandling Baggages')
        carrier_names, values, month, year = get_table_values_monthly(filename, operating_pages[0])
        # Creating DataFrame

        df = pd.DataFrame(values)
        df.columns = features_dict['mishandled_baggage']
        df['Carrier'] = carrier_names
        df['Month'] = month
        df['Year'] = year
        
        df_mb = pd.concat([df_mb, df], axis=0)
        
        ## Mishandling Wheelchairs and Scooters ##
        print('Processing... Mishandling W/S')
        carrier_names, values, month, year = get_table_values_monthly(filename, operating_pages[1])
        # Creating DataFrame

        df = pd.DataFrame(values)
        df.columns = features_dict['mishandled_ws']
        df['Carrier'] = carrier_names
        df['Month'] = month
        df['Year'] = year
        
        df_ws = pd.concat([df_ws, df], axis=0)
        
        ## Denied Boarding ##
        print('Processing... Denied Boarding')
        carrier_names, values, quarter = get_table_values_quarterly(filename, operating_pages[2])
        # Creating DataFrame

        df = pd.DataFrame(values)
        df.columns = features_dict['denied_boarding']
        df['Carrier'] = carrier_names
        df['Quarter'] = quarter
        df['Year'] = year
        
        df_db = pd.concat([df_db, df], axis=0)

Connection Successful!
https://www.transportation.gov/individuals/aviation-consumer-protection/air-travel-consumer-reports-2020
Connection Successful!
https://www.transportation.gov/individuals/aviation-consumer-protection/december-2020-air-travel-consumer-report
Connection Successful!
https://www.transportation.gov/individuals/aviation-consumer-protection/november-2020-air-travel-consumer-report
Connection Successful!
https://www.transportation.gov/individuals/aviation-consumer-protection/october-2020-air-travel-consumer-report
Connection Successful!
https://www.transportation.gov/individuals/aviation-consumer-protection/september-2020-air-travel-consumer-report
Connection Successful!
https://www.transportation.gov/individuals/aviation-consumer-protection/august-2020-air-travel-consumer-report
Connection Successful!
https://www.transportation.gov/individuals/aviation-consumer-protection/july-2020-air-travel-consumer-report
Connection Successful!
https://www.transportation.gov/individu

ValueError: Length mismatch: Expected axis has 0 elements, new values have 4 elements

In [148]:
df_mb.groupby(['Month', 'Carrier'])